# Test env

In [1]:

#imports
import os
import streamlit as st
from PIL import Image
from loguru import logger
# local imports
from ingest.ingester import Ingester
from query.querier import Querier
from summarize.summarizer import Summarizer
import settings
import utils as ut
from query.querier import EnumMode
from ingest.ingester import IngestionMode
from datetime import datetime
import kamervragenEvaluation



In [2]:
SYSTEM_PROMPT= """
### OBJECTIVE ###
Je bent een assistent voor de rijksoverheid. Jouw taak is om vragen te beantwoorden in het Nederlands. Zorg ervoor dat je alleen antwoord geeft op basis van de beschikbare context en dat je daar ook naar verwijst in je antwoord.

### AUDIENCE ###
De doelgroep van jouw antwoorden zijn ambtenaren. Geef alle relevante informatie uit de context, antwoord in het Nederlands leg in maximaal 100 woorden zoveel mogelijk uit.

### GUARDRAILS ###
Indien de context onvoldoende informatie bevat om de vraag te beantwoorden, verzin dan geen informatie maar geef aan dat er onvoldoende informatie beschikbaar is.

### INSTRUCTIONS ###
- Beantwoord de vraag altijd in het Nederlands, zelfs als de context in het Engels is gesteld.
- Vermijd het herhalen van de vraag in het antwoord en het herhalen van de instructies. Voer de instructies uit en geef een concreet antwoord op de gestelde vraag.
- Geef een stapsgewijze redenering bij het beantwoorden van de vraag en refereer naar specifieke zinnen uit de context die hebben bijgedragen aan het antwoord.
- Houd je antwoord nauw verbonden met de context en vermijd het toevoegen van informatie die niet expliciet in de context wordt vermeld.

- Voor meer informatie over de context, zeg het bestandsnaam die gevonden is in de source_document. Mits deze beschikbaar is.
### QUESTION ### \n
"""

In [3]:
# Commented the settings that arent used as parameters in the functions

# DOC_DIR = "./docs"
# CHUNK_DIR = "./chunks"
# VECDB_DIR = "./vector_stores"
# EVAL_DIR = "./evaluate"
# EVAL_APP_HEADER = "Evaluation"
# EVAL_APP_INFO = "./info/evaluation_explanation.txt"
# EVAL_FILE_NAME = "eval.json"
# CHAIN_VERBOSITY = False
LLM_TYPE = "local_llm"
LLM_MODEL_TYPE = "gemma2"
# API_URL = "http://127.0.0.1:11434"
AZUREOPENAI_API_VERSION = "2023-08-01-preview"
EMBEDDINGS_PROVIDER = "local_embeddings"
EMBEDDINGS_MODEL = "textgain/allnli-GroNLP-bert-base-dutch-cased"
TEXT_SPLITTER_METHOD = "NLTKTextSplitter"
# CHAIN_NAME = "conversationalretrievalchain"
# CHAIN_TYPE = "stuff"
# SEARCH_TYPE = "similarity"
# SCORE_THRESHOLD = 0.5
VECDB_TYPE = "chromadb"
CHUNK_SIZE = 1024
# CHUNK_K = 4
CHUNK_OVERLAP = 256
# RETRIEVAL_METHOD = "regular"


folderSelected = "kamerVragen"
my_folder_path_selected, my_vectordb_folder_path_selected = ut.create_vectordb_name(folderSelected)

CONCAT_FILES = True
question_sample_CSV = "question_sample.csv"
VALIDATIONLAPS = 10



In [4]:
SPLITTING_METHODS = [IngestionMode.question_answer,IngestionMode.token_small,IngestionMode.token_medium,IngestionMode.token_large]
CONTEXT_PRESENT= [True, False]
EMBEDDINGS_MODELS = ["GroNLP/bert-base-dutch-cased","textgain/allnli-GroNLP-bert-base-dutch-cased", "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2","dunzhang/stella_en_400M_v5", "actualdata/jina-embeddings-v3"]

In [5]:
querier = None
ingester = None

def init(LLM_TYPE=LLM_TYPE, LLM_MODEL_TYPE=LLM_MODEL_TYPE, EMBEDDINGS_MODEL=EMBEDDINGS_MODEL, EMBEDDINGS_PROVIDER=EMBEDDINGS_PROVIDER, AZUREOPENAI_API_VERSION=AZUREOPENAI_API_VERSION, TEXT_SPLITTER_METHOD=TEXT_SPLITTER_METHOD, CHUNK_SIZE=CHUNK_SIZE, CHUNK_OVERLAP=CHUNK_OVERLAP, VECDB_TYPE=VECDB_TYPE, vectordb_folder= my_vectordb_folder_path_selected, content_folder=my_folder_path_selected):
  # Init
  querier = Querier(
    llm_type=LLM_TYPE, 
    llm_model_type=LLM_MODEL_TYPE, 
    embeddings_model=EMBEDDINGS_MODEL, 
    embeddings_provider=EMBEDDINGS_PROVIDER, 
    azureopenai_api_version=AZUREOPENAI_API_VERSION
    )

  ingester = Ingester(
    collection_name=folderSelected, 
    content_folder=content_folder, 
    vectordb_folder=vectordb_folder,
    embeddings_model=EMBEDDINGS_MODEL,
    text_splitter_method=TEXT_SPLITTER_METHOD,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    vecdb_type=VECDB_TYPE
    )
  return [querier,ingester]
  
# querier,ingester = init()


In [6]:
def ingest(mode=IngestionMode.question_answer_per_page, forceRebuild=True, addedMetaDataURLCSV="docs/metadata.csv", addContext=True):
  ingester.ingest(mode=mode, forceRebuild=forceRebuild, addedMetaDataURLCSV=addedMetaDataURLCSV, addContext=addContext)
# ingest()

In [7]:
def chain(vectorDBPATH = my_vectordb_folder_path_selected):
  querier.make_chain(folderSelected, vectorDBPATH)
# chain()

In [8]:
def write_to_error_log(error, params):
  with open("error_log.txt", "a") as f:
    f.write(f"{datetime.now()} | {params} - {error}\n")

In [9]:

querier,ingester = init(EMBEDDINGS_MODEL=EMBEDDINGS_MODEL, vectordb_folder=my_vectordb_folder_path_selected)
kamervragenEvaluation.create_evaluation_sample_questions(my_folder_path_selected,ingester=ingester, destinationCSV=question_sample_CSV)

/Users/markheijnekamp/Documents/GitHub/LearningLion-kamervragen/utils.py:113: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/opt/homebrew/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/homebrew/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated i

done writing to csv


## Place folder contents in json file

In [ ]:
current_item = 0
total_items = VALIDATIONLAPS * len(SPLITTING_METHODS) * len(CONTEXT_PRESENT) * len(EMBEDDINGS_MODELS)
for embeddingModel in EMBEDDINGS_MODELS:
  for splittingMethod in SPLITTING_METHODS:
    for context in CONTEXT_PRESENT:
      for i in range(VALIDATIONLAPS):
        current_item += 1
        try:
          chunk_size = CHUNK_SIZE
          if splittingMethod == IngestionMode.token_small:
            chunk_size = 128
          elif splittingMethod == IngestionMode.token_medium:
            chunk_size = 512
          elif splittingMethod == IngestionMode.token_large:
            chunk_size = 1024
            
          # Setup
          my_folder_path_selected, my_vectordb_folder_path_selected = ut.create_vectordb_name(folderSelected, chunk_size=chunk_size, chunk_overlap=0, splitting_method=splittingMethod, embeddings_model=embeddingModel, added_context=context)
          querier,ingester = init(EMBEDDINGS_MODEL=embeddingModel, vectordb_folder=my_vectordb_folder_path_selected)
          # Ingestion
          ingest(mode=splittingMethod, addContext=context, addedMetaDataURLCSV="docs/metadata.csv")
          # RAG CHAIN
          chain(vectorDBPATH=my_vectordb_folder_path_selected)
          
          # Evaluation
          kamervragenEvaluation.evaluate_with_sample_questions(
            question_sample_CSV,querier=querier, 
            toCSV=True, 
            ingestionMode=splittingMethod, 
            addedMetaDataURLCSV="docs/metadata.csv", 
            addContext=context,
            embeddings_model=embeddingModel,
            text_splitter_method=TEXT_SPLITTER_METHOD,
            embeddings_provider=EMBEDDINGS_PROVIDER,
            database=VECDB_TYPE,
            concatFiles=CONCAT_FILES,
            )
        except Exception as e:
          print(e)
          # Write error to file
          write_to_error_log(e, f"splittingMethod={splittingMethod}, context={context}, embeddingModel={embeddingModel}")
          continue
        
        
        print(f"done with {current_item} of {total_items}")
  print(f"Done with iteration {time}")

In [ ]:
LongTimeEmbeddingsModels = ["BAAI/bge-multilingual-gemma2", "Alibaba-NLP/gte-Qwen2-7B-instruct","Alibaba-NLP/gte-multilingual-base"]

current_item = 0
total_items = VALIDATIONLAPS * len(SPLITTING_METHODS) * len(CONTEXT_PRESENT) * len(EMBEDDINGS_MODELS)
for embeddingModel in EMBEDDINGS_MODELS:
  for splittingMethod in SPLITTING_METHODS:
    for context in CONTEXT_PRESENT:
      for i in range(VALIDATIONLAPS):
        current_item += 1
        try:
          chunk_size = CHUNK_SIZE
          if splittingMethod == IngestionMode.token_small:
            chunk_size = 128
          elif splittingMethod == IngestionMode.token_medium:
            chunk_size = 512
          elif splittingMethod == IngestionMode.token_large:
            chunk_size = 1024
            
          # Setup
          my_folder_path_selected, my_vectordb_folder_path_selected = ut.create_vectordb_name(folderSelected, chunk_size=chunk_size, chunk_overlap=0, splitting_method=splittingMethod, embeddings_model=embeddingModel, added_context=context)
          querier,ingester = init(EMBEDDINGS_MODEL=embeddingModel, vectordb_folder=my_vectordb_folder_path_selected)
          # Ingestion
          ingest(mode=splittingMethod, addContext=context, addedMetaDataURLCSV="docs/metadata.csv")
          # RAG CHAIN
          chain(vectorDBPATH=my_vectordb_folder_path_selected)
          
          # Evaluation
          kamervragenEvaluation.evaluate_with_sample_questions(
            question_sample_CSV,querier=querier, 
            toCSV=True, 
            ingestionMode=splittingMethod, 
            addedMetaDataURLCSV="docs/metadata.csv", 
            addContext=context,
            embeddings_model=embeddingModel,
            text_splitter_method=TEXT_SPLITTER_METHOD,
            embeddings_provider=EMBEDDINGS_PROVIDER,
            database=VECDB_TYPE,
            concatFiles=CONCAT_FILES,
            )
        except Exception as e:
          print(e)
          # Write error to file
          write_to_error_log(e, f"splittingMethod={splittingMethod}, context={context}, embeddingModel={embeddingModel}")
          continue
        
        
        
        print(f"done with {current_item} of {total_items}")
  print(f"Done with iteration {time}")

# Re test items

In [10]:
# add context to question_sample.csv
import pandas as pd

from ingest.file_parser import FileParser
from ingest.ingest_utils import IngestUtils

df = pd.read_csv(question_sample_CSV)
file_parser = FileParser()
ingestutils = IngestUtils(CHUNK_SIZE, 0, None, TEXT_SPLITTER_METHOD)
if 'context' not in df.columns:
  for index, row in df.iterrows():
    # get file
    file = row["Filename"]
    if file.endswith(".pdf"):
      file_path = os.path.join(my_folder_path_selected, file)
      raw_pages, metadata = file_parser.parse_file(file_path)
      documents = ingestutils.clean_text_to_docs(raw_pages, metadata)
      introduction = documents[0].page_content.split("vraag")[0]

    df.at[index, "context"] = introduction
    df.to_csv(question_sample_CSV, index=False)


In [ ]:
current_item = 0
total_items = VALIDATIONLAPS * len(SPLITTING_METHODS) * len(CONTEXT_PRESENT) * len(EMBEDDINGS_MODELS)
for embeddingModel in EMBEDDINGS_MODELS:
  for splittingMethod in SPLITTING_METHODS:
    for context in CONTEXT_PRESENT:
        for i in range(VALIDATIONLAPS):
          current_item += 1
          try:
            chunk_size = CHUNK_SIZE
            if splittingMethod == IngestionMode.token_small:
              chunk_size = 128
            elif splittingMethod == IngestionMode.token_medium:
              chunk_size = 512
            elif splittingMethod == IngestionMode.token_large:
              chunk_size = 1024
              
            # Setup
            my_folder_path_selected, my_vectordb_folder_path_selected = ut.create_vectordb_name(folderSelected, chunk_size=chunk_size, chunk_overlap=0, splitting_method=splittingMethod, embeddings_model=embeddingModel, added_context=context)
            querier,ingester = init(EMBEDDINGS_MODEL=embeddingModel, vectordb_folder=my_vectordb_folder_path_selected)
            # RAG CHAIN
            chain(vectorDBPATH=my_vectordb_folder_path_selected)
            
            # Evaluation
            kamervragenEvaluation.evaluate_with_sample_questions(
              question_sample_CSV,querier=querier, 
              toCSV=True, 
              ingestionMode=splittingMethod, 
              addedMetaDataURLCSV="docs/metadata.csv", 
              addContext=context,
              embeddings_model=embeddingModel,
              text_splitter_method=TEXT_SPLITTER_METHOD,
              embeddings_provider=EMBEDDINGS_PROVIDER,
              database=VECDB_TYPE,
              concatFiles=CONCAT_FILES,
              )
          except Exception as e:
            print(e)
            # Write error to file
            write_to_error_log(e, f"splittingMethod={splittingMethod}, context={context}, embeddingModel={embeddingModel}")
            continue
          
          
          
          print(f"done with {current_item} of {total_items}")

## Evaluations

In [11]:
import kamervragenEvaluation


### Check if one single file can be retrived

In [ ]:

kamervragenEvaluation.test_retrival(
  my_folder_path_selected, 
  ingester, 
  querier=querier, 
  toCSV=True,
  ingestionMode=IngestionMode.question_answer_per_page, 
  addedMetaDataURLCSV="docs/metadata.csv", 
  addContext=True,
  embeddings_model=EMBEDDINGS_MODEL,
  text_splitter_method=TEXT_SPLITTER_METHOD,
  embeddings_provider=EMBEDDINGS_PROVIDER,
  database=VECDB_TYPE,
  ConcatFiles=CONCAT_FILES
  )

In [ ]:
kamervragenEvaluation.store_questions_and_answers_CSV(my_folder_path_selected, ingester,concatFiles=CONCAT_FILES)

In [ ]:
kamervragenEvaluation.test_retrival_map_grading(
  my_folder_path_selected, 
  ingester, 
  querier=querier, 
  toCSV=True,
  ingestionMode=IngestionMode.question_answer_per_page, 
  addedMetaDataURLCSV="docs/metadata.csv", 
  addContext=True,
  embeddings_model=EMBEDDINGS_MODEL,
  text_splitter_method=TEXT_SPLITTER_METHOD,
  embeddings_provider=EMBEDDINGS_PROVIDER,
  database=VECDB_TYPE,
  concatFiles=CONCAT_FILES)